In [2]:
import numpy as np

In [3]:
# load the mnist dataset

from dataset import fetch_mnist
X_train, Y_train, X_test, Y_test = fetch_mnist()

In [4]:
from tinygrad import Tensor, nn
class TinyMNIST:
    def __init__(self):
        self.l1 = nn.Linear(784, 128, bias=True)
        self.l2 = nn.Linear(128, 10, bias=True)

    def __call__(self, x):
        x = x.flatten(1)
        x = self.l1(x)
        x = x.relu()
        x = self.l2(x)
        return x



In [8]:
import os
os.environ['GPU'] = '1'

model = TinyMNIST()
optim = nn.optim.SGD([model.l1.weight, model.l2.weight], lr=0.001)

BS = 32
with Tensor.train():
    for i in range(400):
        samp = np.random.randint(0, X_train.shape[0], size=(BS, 1))
        ipt = Tensor(X_train[samp]).float()
        samp = samp.reshape(BS)
        target = Tensor(Y_train[samp]).float()
        optim.zero_grad()
        out = model(ipt)
        loss = out.sparse_categorical_crossentropy(target)
        loss.backward()
        optim.step()

AttributeError: module 'gpuctypes.cuda' has no attribute 'nvrtcCreateProgram'

In [ ]:
corr = 0
samples = 1000
for i in range(samples):
    samp = np.random.randint(0, X_test.shape[0], size=(1, 1))
    X = Tensor(X_test[samp]).float()
    Y = Y_test[samp][0][0]
    guess = model(X).argmax()
    if Y == guess.item(): corr += 1
print(f"accuracy: {corr/samples*100}%")

100%|██████████| 10000/10000 [00:00<00:00, 15935.41it/s]

accuracy: 91.23%


In [ ]:
#torch.save(model.state_dict(), "models/mnist_1.pt")